In [ ]:
#将轨道系数导出到orb_coeff.t
import os
import numpy as np
os.chdir('C:/Users/wanli/Desktop/ML/reorganization/dnn-coul/xyz-orb')

def orb33(file):
    data=np.loadtxt(file)
    data =data.tolist()
    if len(data) < 33:
        for i in range(33-len(data)):
            data.append(0)
    return data
        
        
outFiles = []
for filename in os.listdir('./'): 
    if filename.endswith('.out'):
        outFiles.append(filename)

orb_coeff = []
for i in range(len(outFiles)):
    orb_coeff.append(orb33(outFiles[i]))
torch.save(torch.Tensor(orb_coeff),'orb_coeff.t')

In [1]:
import torch
from torch.utils.data import TensorDataset,random_split
import pandas as pd
from rdkit import Chem,DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import ChemicalFeatures
from rdkit import RDConfig
import os
from dgllife.data import MoleculeCSVDataset
from functools import partial
from dgllife.utils import smiles_to_bigraph, ConsecutiveSplitter

# Obtain the features of atoms and bonds
def load_coeff(mol):
    mol = Chem.MolToSmiles(mol, canonical=True)
    df = pd.read_csv('train_set_5631.csv')

    sms=[Chem.MolToSmiles(Chem.MolFromSmiles(sm), canonical=True) for sm in df['smiles'].tolist()]
    idx=sms.index(mol)
    
    coeff = torch.load('orb_coeff.t')
    return coeff[idx]


def featurize_atoms(mol):  
    feats = []
    for atom in mol.GetAtoms():
        hy = [int(atom.GetHybridization()==y) for y in [Chem.rdchem.HybridizationType.SP,
              Chem.rdchem.HybridizationType.SP2,Chem.rdchem.HybridizationType.SP3]]
        feats.append([atom.GetAtomicNum(), atom.GetExplicitValence(), atom.GetImplicitValence(),
                      atom.GetTotalNumHs(), atom.GetFormalCharge(), atom.GetNumRadicalElectrons(),
                      atom.GetDegree(), int(atom.GetIsAromatic()), int(atom.IsInRing())]+hy)
        
    return {'atomic': torch.tensor(feats)}


def featurize_edges(mol, add_self_loop=True):     
    feats = []
    coo = load_coeff(mol)
    num_atoms = mol.GetNumAtoms()
    for i in range(num_atoms):
        for j in range(num_atoms):
            e_ij = mol.GetBondBetweenAtoms(i,j)
            if e_ij is None:
                bond_type = None
            else:
                bond_type = e_ij.GetBondType()
                feats.append([float(bond_type == x)for x in (None, Chem.rdchem.BondType.SINGLE,
                              Chem.rdchem.BondType.DOUBLE, Chem.rdchem.BondType.TRIPLE, Chem.rdchem.BondType.AROMATIC)]
                             +[coo[i] * coo[j]])
    return {'edgic': torch.tensor(feats)}


if __name__ == "__main__":

    df = pd.read_csv('train_set_5631.csv')
   
    # SMILES to graph-based dataset for prediction model with DGL-Life
    dataset=MoleculeCSVDataset(df=df,
                               smiles_to_graph=partial(smiles_to_bigraph, add_self_loop=False),
                               node_featurizer=featurize_atoms,
                               edge_featurizer=None,#featurize_edges,
                               smiles_column='smiles',
                               cache_file_path='graph.pt',log_every=1000)
#    print(dataset)
    train_set, val_set, test_set = ConsecutiveSplitter.train_val_test_split(dataset, frac_train=0.8, frac_val=0.1, frac_test=0.1)
    torch.save([train_set,val_set,test_set], "opv_graph-onlyatom.pt")


    

Using backend: pytorch


Processing dgl graphs from scratch...
Processing molecule 1000/5631
Processing molecule 2000/5631
Processing molecule 3000/5631
Processing molecule 4000/5631
Processing molecule 5000/5631


In [3]:
#featurize_atoms(Chem.MolFromSmiles('c1cnc2c(c1)Cc1c2ccc2c3[nH]ccc3[nH]c21'))
#featurize_edges(Chem.MolFromSmiles('c1cnc2c(c1)Cc1c2ccc2c3[nH]ccc3[nH]c21'))

In [3]:
dataset.load_full = True
dataset[0]

('c1cc2[nH]c3c4ncc5ccCc5c4Cc3c2o1',
 Graph(num_nodes=18, num_edges=44,
       ndata_schemes={'atomic': Scheme(shape=(13,), dtype=torch.float32)}
       edata_schemes={'edgic': Scheme(shape=(6,), dtype=torch.float32)}),
 tensor([246.9197]),
 tensor([1.]))

In [35]:
g = smiles_to_bigraph('c1cc2[nH]c3c4ncc5ccCc5c4Cc3c2o1')
print(g)

Graph(num_nodes=18, num_edges=44,
      ndata_schemes={}
      edata_schemes={})


In [2]:

www=featurize_edges(Chem.MolFromSmiles('c1cc2[nH]c3c4ncc5ccCc5c4Cc3c2o1'))

print([i for i in www.values()][0].size())

0
1
2
3
4
5
torch.Size([44, 6])


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn.utils import weight_norm
from dgllife.model.gnn.gat import GAT
from dgl.readout import softmax_nodes,sum_nodes


class ConvLayer(nn.Module):

    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2, model='Gen'):
        super(ConvLayer, self).__init__()
        self.conv = weight_norm(nn.Conv1d(n_inputs, (n_outputs * 2), kernel_size, stride=stride,
                                          padding=padding,dilation=dilation))
        self.padding = padding
        self.model = model
        self.glu = nn.GLU(dim=1)
        self.dropout = nn.Dropout(dropout)
        self.trans = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.init_weights()

    def init_weights(self):
        torch.nn.init.xavier_uniform_(self.conv.weight)
        if self.trans is not None:
            torch.nn.init.xavier_uniform_(self.trans.weight)

    def forward(self, x):
        y = self.conv(x)
        out = self.glu(y[:, :, :-self.padding].contiguous()) if self.model == 'Gen' else self.glu(y)
        out = self.dropout(out)
        if self.trans is not None:
            x = self.trans(x)
        return out + x


class Encoder(nn.Module):

    def __init__(self, input_size, hid_size, n_levels, kernel_size=3, dropout=0.2, model='Gen'):
        super(Encoder, self).__init__()
        layers = []
        for i in range(n_levels):
            dilation_size = 2 ** i
            padding = (kernel_size - 1) * dilation_size if model == 'Gen' else dilation_size
            if i == 0:
                layers += [ConvLayer(input_size, hid_size, kernel_size, stride=1, dilation=dilation_size,
                                     padding=padding, dropout=dropout, model=model)]
            else:
                layers += [ConvLayer(hid_size, hid_size, kernel_size, stride=1, dilation=dilation_size,
                                     padding=padding, dropout=dropout, model=model)]
        
        self.network = (nn.Sequential)(*layers)

    def forward(self, x):
        return self.network(x)


class NNet(nn.Module):

    def __init__(self, n_in, n_out, hide=(64, 64, 8)):
        super(NNet, self).__init__()
        self.n_hide = len(hide)
        self.fcs = nn.ModuleList([weight_norm(nn.Linear(n_in, hide[i])) if i == 0 else 
                                  weight_norm(nn.Linear(hide[(i - 1)], n_out)) if i == self.n_hide else 
                                  weight_norm(nn.Linear(hide[(i - 1)], hide[i])) for i in range(self.n_hide + 1)])
        self.init_weights()

    def init_weights(self):
        for i in range(self.n_hide + 1):
            self.fcs[i].weight.data.normal_(0, 0.01)

    def forward(self, x):
        for i in range(self.n_hide):
            x = F.relu(self.fcs[i](x))
        x = self.fcs[(-1)](x)
        return x


class GEN(nn.Module):

    def __init__(self, n_props, dic_size, emb_size, hid_size=256, n_levels=5, kernel_size=3, dropout=0.2):
        super(GEN, self).__init__()
        self.emb = nn.Embedding(dic_size, emb_size, padding_idx=0)
        self.dense0 = NNet(n_props, 16, hide=(8, 64))
        self.encoder = Encoder((emb_size + 16), hid_size, n_levels, kernel_size, dropout=dropout, model='Gen')
        self.decoder = nn.Linear(hid_size, dic_size)

    def forward(self, input, label):
        emb = self.emb(input)
        pro = self.dense0(label).unsqueeze(-1).expand(-1, -1, emb.size(1))
        y = self.encoder(torch.cat((emb.transpose(1, 2), pro), dim=1))
        o = self.decoder(y.transpose(1, 2))
        return o.contiguous()


class GlobalAttentionPooling(nn.Module):
    
    def __init__(self, gate_nn, feat_nn=None):
        super(GlobalAttentionPooling, self).__init__()
        self.gate_nn = gate_nn
        self.feat_nn = feat_nn
        self.gate, self.feat = (None, None)

    def forward(self, graph, feat):
        
        with graph.local_scope():
            gate = self.gate_nn(feat)
            assert gate.shape[-1] == 1, "The output of gate_nn should have size 1 at the last axis."
            feat = self.feat_nn(feat) if self.feat_nn else feat

            graph.ndata['gate'] = gate
            gate = softmax_nodes(graph, 'gate')
            graph.ndata.pop('gate')

            self.gate, self.feat = gate, feat 

            graph.ndata['r'] = feat * gate
            readout = sum_nodes(graph, 'r')
            graph.ndata.pop('r')

            return readout


class PRE(torch.nn.Module):

    def __init__(self, h_size, emb_h, dim):
        super(PRE, self).__init__()

        self.emb_h = NNet(h_size,emb_h,hide=(64,128,64))
        self.convs = GAT(emb_h,hidden_feats=[dim,dim*2,dim*2,dim*2,dim*2,dim],num_heads=[8,4,4,4,4,1],
                         agg_modes=['flatten','flatten','flatten','flatten','flatten','mean'])

        self.global_atten = GlobalAttentionPooling(NNet(dim, 1, hide=(32,)), NNet(dim, 1, hide=(32,)))

    def forward(self, bg, feats):
        feats = self.emb_h(feats.float())
        feats = self.convs(bg,feats)
        x = self.global_atten(bg,feats)
        return x

    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    

In [ ]:
import argparse
import config
import time
import torch
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from model import PRE
from torch.utils.tensorboard import SummaryWriter
import dgl
from gpuinfo import GPUInfo
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from scipy.stats import pearsonr


def train(train_iter):
    model.train()
    total_loss = 0
    for data in train_iter:
        smiles, bg, labels, masks = data
        bg, labels, masks = bg.to(device), labels[:,args.property_n].to(device), masks.to(device)#
        node_feats = bg.ndata.pop('atomic').to(device)
        # edge_feats = bg.edata.pop('e').to(device)
        
        optimizer.zero_grad()
        outputs = model(bg, node_feats)
        loss = (F.mse_loss(outputs,labels)* (masks != 0).float()).mean()
        loss.backward()
        total_loss += loss.item()
        optimizer.step()  

    return total_loss/len(train_iter)


def evaluate(data_iter):
    with torch.no_grad():
        model.eval()
        total_loss = 0
        for data in data_iter:
            smiles, bg, labels, masks = data
            bg, labels, masks = bg.to(device), labels[:,args.property_n].to(device), masks.to(device)#
            node_feats = bg.ndata.pop('atomic').to(device)
            # edge_feats = bg.edata.pop('e').to(device)
            
            outputs = model(bg, node_feats)
            loss = (F.mse_loss(outputs,labels)* (masks != 0).float()).mean()
            total_loss += loss.item()

    return total_loss/len(data_iter)


def collate_molgraphs(data):

    smiles, graphs, labels, masks = map(list, zip(*data))

    bg = dgl.batch(graphs)
    bg.set_n_initializer(dgl.init.zero_initializer)
    bg.set_e_initializer(dgl.init.zero_initializer)
    labels = torch.stack(labels, dim=0)

    if masks is None:
        masks = torch.ones(labels.shape)
    else:
        masks = torch.stack(masks, dim=0)

    return smiles, bg, labels, masks



if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Prediction Modeling',parents=[config.parser])
    parser.add_argument('--save_name', type=str, default='reo_pre0.pt',help='the name of save model')
    args = parser.parse_args()
    print(args)

    torch.manual_seed(1024)
    torch.cuda.manual_seed(1024)

    train_set, vali_set, test_set = torch.load("data/opv_graph-s.pt")
    train_iter = DataLoader(train_set, args.batch_size, shuffle=True, collate_fn=collate_molgraphs)
    val_iter = DataLoader(vali_set, args.batch_size, shuffle=False, collate_fn=collate_molgraphs)
    test_iter = DataLoader(test_set,1, shuffle=False, collate_fn=collate_molgraphs)

    model = PRE(h_size=args.h_size,emb_h=args.emb_h,dim=args.hid_size)
    # model_pre = torch.load("init_model/results/lumo_pre0.pt").state_dict()
    # model.load_state_dict(model_pre, strict=False)

    # if torch.cuda.is_available():
    #     available_device=GPUInfo.check_empty()
    #     percent,memory=GPUInfo.gpu_usage()
    #     # min_memory=memory.index(min([memory[i] for i in available_device]))
    #     min_percent=percent.index(min([percent[i] for i in available_device]))
    #     torch.cuda.set_device(min_percent)#min_memory

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    optimizer = getattr(optim, args.optim)(model.parameters(), lr=args.lr)
    
    best_vloss =1000
    writer = SummaryWriter()

    for epoch in range(1,args.epochs+1):
        start_time = time.time()
        train_loss = train(train_iter)
        val_loss = evaluate(val_iter)

        print('-' * 80)
        print('epoch: {:4d} | time: {:4.4f}s | train loss: {:4.6f} | valid loss: {:4.6f}'.format
              (epoch, time.time() - start_time, train_loss, val_loss))

        writer.add_scalar('Train Loss', train_loss, epoch)
        writer.add_scalar('Valid Loss', val_loss, epoch)
        
        if val_loss < best_vloss:
            print('-' * 80)
            print('Save model!')
            torch.save(model, 'results/'+args.save_name)
            best_vloss = val_loss

    writer.close()

    model = torch.load("results/"+args.save_name)
    test_loss = evaluate(test_iter)
    print('=' * 40)
    print('End of training | test MSE {:4.6f}'.format(test_loss))
    print('=' * 40)


    with torch.no_grad():
        model.eval()
        total_loss = 0
        Targets = []
        Outputs = []
        for data in test_iter:
            smiles, bg, labels, masks = data
            bg, labels, masks = bg.to(device), labels[:,args.property_n].to(device), masks.to(device)#
            node_feats = bg.ndata.pop('atomic')
            # edge_feats = bg.edata.pop('e').to(device)
            
            outputs = model(bg, node_feats.to(device))
            loss = (F.l1_loss(outputs,labels)*(masks != 0).float()).mean()
            total_loss += loss.item()
            Outputs += [outputs.item()]
            Targets += [labels.item()]

    print('Test MAE {:4.6f} | REO_pearsonr {:4.6f}'.format(total_loss/len(test_iter),pearsonr(Targets, Outputs)[0]))

    plt.figure(figsize=(5, 5),dpi=500)
    plt.scatter(Targets,Outputs)
    plt.axis([0.2,0.7, 0.2,0.7])
    plt.title('reorganization energy',fontsize=12)
    plt.xlabel('reo_real(eV)',fontsize=10)
    plt.ylabel('reo_pre(eV)',fontsize=10)
    plt.savefig("re0.jpg")
